In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!rm -r "/content/sample_data/train"
!rm -r "/content/sample_data/valid"
!rm "/content/sample_data/flowers_jpg.zip"
!cp "/content/drive/My Drive/flowers_jpg/flowers_jpg.zip" "/content/sample_data/"
!unzip "/content/sample_data/flowers_jpg.zip" -qq -d /content/sample_data/
!ls -1 /content/sample_data/train/JPEG | wc -l

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.cuda.is_available()

True

In [0]:
dev = torch.device('cuda')

In [0]:

#torch.cuda.memory_cached(dev), torch.cuda.memory_allocated(dev)

In [0]:
import os
# train_ds = datasets.MNIST(root='./mnist/', train=True, transform=transforms.ToTensor(), download=True)
# validation_ds = datasets.MNIST(root='./mnist/', train=False, transform=transforms.ToTensor(), download=False)
data_folder  = '/content/sample_data'
train_dir = os.path.join(data_folder, "train")
valid_dir = os.path.join(data_folder, "valid")
#check_dir = os.path.join(data_folder, "check")

train_transforms = transforms.Compose([
    transforms.RandomRotation(15),
    transforms.Resize((240, 360)),  #(240, 360)
    #transforms.CenterCrop(224),
    # 
    # transforms.RandomResizedCrop(size=224),
    transforms.RandomHorizontalFlip(),
    # transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

validation_transforms = transforms.Compose([
    transforms.Resize((240, 360)),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
validation_dataset = datasets.ImageFolder(valid_dir, transform=validation_transforms)
#check_dataset = datasets.ImageFolder(check_dir, transform=validation_transforms)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, num_workers = 1, pin_memory = True, shuffle = True)
validation_loader = DataLoader(dataset=validation_dataset, batch_size=BATCH_SIZE, num_workers = 1, pin_memory = True, shuffle = True)
#check_loader = DataLoader(dataset=check_dataset, batch_size=BATCH_SIZE, num_workers = 4, pin_memory = False, shuffle = True)

In [0]:
def init_weights(m):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.zero_()


class VAE(nn.Module):
    def __init__(self, hid_dim):
        super(VAE, self).__init__()
        # flatsize = 2304 #input_dim * 16 // 4
        
        #rs = models.wide_resnet50_2(pretrained=True)
        #modules = list(rs.children())[:-1]      # delete the last fc layer.
        #self.resnet = nn.Sequential(*modules)
        self.resnet = torch.hub.load('pytorch/vision:v0.5.0', 'resnext50_32x4d', pretrained=True)
        
        for param in self.resnet.parameters():
            param.requires_grad = False
        
        # self.resnet.layer4.requires_grad = True
        # for param in self.resnet.layer4.parameters():
        #     param.requires_grad = True
        
        # self.resnet.avgpool.requires_grad = True
        # for param in self.resnet.avgpool.parameters():
        #    param.requires_grad = True
            
        #ii = 0
        
        #for i,param in enumerate(self.resnet.parameters()):
        #    if (i<=140):
        #        param.requires_grad = False
        #    ii =i
        #print(ii)""

        #self.resnet.fc = nn.Flatten()
        
        self.resnet.fc = nn.Sequential(
                      nn.Flatten(),
                      nn.Linear(in_features=2048, out_features=1024, bias=True),
                      nn.BatchNorm1d(1024, momentum=0.01), #nn.ReLU(),
                      nn.LeakyReLU(0.1, True),#nn.ReLU()
                      nn.Dropout(0.4)
        )
        
        self.resnet.fc.apply(init_weights)
        
        self.enc_dense_mu = nn.Linear(1024, hid_dim)
        self.enc_dense_std = nn.Linear(1024, hid_dim)
        init_weights(self.enc_dense_mu)
        init_weights(self.enc_dense_std)
           
        self.dec_layer_linear = nn.Sequential(
            nn.Linear(hid_dim, 1024),
            nn.BatchNorm1d(1024),
            #nn.Tanh(),
            nn.LeakyReLU(0.1, True),
            nn.Dropout(0.3),
            nn.Linear(1024, 25088),
            nn.BatchNorm1d(25088),
            nn.LeakyReLU(0.1, True),
            nn.Dropout(0.3),
        )
        
        self.dec_layer_linear.apply(init_weights)
        
        self.dec_convTrans = nn.Sequential(
            nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=(3,3), stride=(2,2),padding=(0,0)),
            nn.BatchNorm2d(256, momentum=0.01),
            nn.LeakyReLU(0.01, True),
            nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=(3,3), stride=(2,2),padding=(0,0)),
            nn.BatchNorm2d(128, momentum=0.01),
            nn.LeakyReLU(0.01, True),
            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=(3,3), stride=(2,3),padding=(0,0)),
            nn.BatchNorm2d(64, momentum=0.01),
            nn.LeakyReLU(0.01, True),
            nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=(3,3), stride=(2,2),padding=(0,0)),
            nn.BatchNorm2d(32, momentum=0.01),
            nn.LeakyReLU(0.01, True),
            nn.ConvTranspose2d(in_channels=32, out_channels=3, kernel_size=(3,3), stride=(2,2),padding=(0,0)),
            nn.BatchNorm2d(3, momentum=0.01),
            nn.Sigmoid(),
            nn.Upsample(size=(240,360), mode='bilinear', align_corners=True)
            #nn.Sigmoid() # убрать странные kernel, оставить только 2:3 + Upsample bicubic
        )
        
        #self.dec_convTrans = nn.Sequential(
        #    nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=(3,3), stride=(2,2),padding=(0,0)),
        #    nn.BatchNorm2d(256, momentum=0.01),
        #    nn.LeakyReLU(0.01, True),
        #    nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=(3,3), stride=(2,2),padding=(0,0)),
        #    nn.BatchNorm2d(128, momentum=0.01),
        #    nn.LeakyReLU(0.01, True),
        #    nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=(3,3), stride=(2,2),padding=(0,0)),
        #    nn.BatchNorm2d(128, momentum=0.01),
        #    nn.LeakyReLU(0.01, True),
        #    nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=(3,3), stride=(2,2),padding=(0,0)),
        #    nn.BatchNorm2d(64, momentum=0.01),
        #    nn.LeakyReLU(0.01, True),
        #    nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=(7,7), stride=(2,2),padding=(0,0)),
        #    nn.BatchNorm2d(3, momentum=0.01),
        #    nn.Sigmoid()
        #   nn.Upsample(size=(224,224), mode='bilinear', align_corners=True)
        #)
        
        self.dec_convTrans.apply(init_weights)
        
    def encoder(self, x):
        x1 = self.resnet(x)  # ResNet
        #print('encoder resnet nan ', torch.isnan(x).any(), ' inf ', torch.isinf(x).any())
        #x = x.view(x.size(0), -1)  # flatten output of conv
        #x1 = self.enc_layer(x1)
        #print('encoder linear nan ', torch.isnan(x1).any(), ' inf ', torch.isinf(x1).any())
        # x1 = self.enc_avgpool(x1)
        # x1 = self.enc_linear(x1)
        return self.enc_dense_mu(x1), self.enc_dense_std(x1)
          
    def decoder(self, z):
        z1 = self.dec_layer_linear(z).view(-1, 512, 7, 7)
        #print('decoder linear nan ', torch.isnan(z1).any(), ' inf ', torch.isinf(z1).any())
        z1 = self.dec_convTrans(z1)
        #print('dec img ',z1.size())
        #z1 = F.interpolate(z1, size=(224, 224), mode='bilinear')
        return z1
    
    def sample(self, mu, std):
        std = torch.exp(0.5 * std)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)
    
    def forward(self, x):
        mu, std = self.encoder(x) #x.view(-1, 784)
        #print('mu nan ', torch.isnan(mu).any(), ' inf ', torch.isinf(mu).any())
        #print('std nan ', torch.isnan(std).any(), ' inf ', torch.isinf(std).any())
        z = self.sample(mu, std)
        #print('sample nan ', torch.isnan(z).any(), ' inf ', torch.isinf(z).any())
        d = self.decoder(z)
        #print('decoded nan ', torch.isnan(d).any(), ' inf ', torch.isinf(d).any())
        return d, mu, std

In [0]:
vae = VAE(256) #512?
# vae.apply(init_weights)
vae.load_state_dict(torch.load('/content/drive/My Drive/flowers_jpg/flower_3x3upsample.torch.vae'))  #,map_location=torch.device('cpu')
vae = vae.to(dev)
#vae.eval()

Downloading: "https://github.com/pytorch/vision/archive/v0.5.0.zip" to /root/.cache/torch/hub/v0.5.0.zip
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:01<00:00, 55.2MB/s]


In [0]:
import numpy as np

class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        for im in tensor:
            for t, m, s in zip(im, self.mean, self.std):
                t.mul_(s).add_(m)
        return tensor

unorm = UnNormalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

def loss_function(pred, x, mu, std):
    x1 = unorm(x)
    BCE = F.binary_cross_entropy(pred, x, reduction='sum') #.view(-1,150528) # ?? перевести на view ?
    KLD = -0.5 * torch.sum(1 + std - mu.pow(2) - std.exp())
    return BCE + KLD #, BCE, KLD

In [0]:
from tqdm import tqdm_notebook as tqdm
import numpy as np
def train(model, dataloader, dataloader_test, num_epoch=100, init_lr=0.001, lgamma=0.99):
    losses = []
    scores = []
    # gd = torch.optim.SGD(model.parameters(),lr=init_lr,momentum=0.9) 
    gd = torch.optim.Adam(model.parameters(),lr=init_lr,amsgrad=True)
    
    train_losses = []
    test_losses = []
    
    scheduler = torch.optim.lr_scheduler.StepLR(gd, step_size=1, gamma=lgamma)
    
    for epoch in range(num_epoch):
        model.train(True)
        train_loss = 0
        for i, (batch, _) in tqdm(enumerate(dataloader)):
            #print(batch.shape)
            #print()
            # print('batch nan ', torch.isnan(batch).any(), ' inf ', torch.isinf(batch).any())
            batch_c = batch.to(dev)
            gd.zero_grad()
            pred_batch, mu, log_var = model(batch_c)
            #print(pred_batch.size())
            #break
            # print('neg values ', torch.lt(pred_batch,0.0).any())
            #print(pred_batch.size(), batch.size())
            #print('predicted nan ', torch.isnan(pred_batch).any(), ' inf ', torch.isinf(pred_batch).any())
            loss = loss_function(pred_batch, batch_c, mu, log_var)
            (loss).backward()
            train_loss += loss.item()
            #print('BCE ', bcei.item(),'KLD ',kldi.item(),' train loss ',train_loss,
            #      'min ', torch.min(pred_batch).cpu().detach().numpy(), 'max ', torch.max(pred_batch).cpu().detach().numpy(),
            #      'min ', torch.min(batch_c).cpu().detach().numpy(), 'max ', torch.max(batch_c).cpu().detach().numpy())
            losses.append(loss.data.cpu().numpy())
            gd.step()
            gd.zero_grad()
        train_mse = np.mean(losses[-(i+1):])
        scheduler.step()
        for param_group in gd.param_groups:
            print('LR: ', param_group['lr'])
        
        #break
        
        model.train(False)
        test_loss = 0
        for i, (batch, _) in tqdm(enumerate(dataloader_test)):
            batch_c = batch.to(dev)
            pred_batch, mu, log_var = model(batch_c)
            loss = loss_function(pred_batch, batch_c, mu, log_var)
            test_loss += loss.item()
        
        avg_train_loss = train_loss / len(dataloader.dataset)
        avg_test_loss = test_loss / len(dataloader_test.dataset)
        
        train_losses.append(avg_train_loss)
        test_losses.append(avg_test_loss)
        torch.save(model.state_dict(), '/content/drive/My Drive/flowers_jpg/flower_3x3upsample.torch.vae')
        print("Epoch: {}, Train loss: {}, Test loss: {}".format(epoch + 1, 
                                                                avg_train_loss,
                                                                avg_test_loss))
        
    
    return train_losses, test_losses

In [0]:
vae.resnet.layer4.requires_grad = True
for param in vae.resnet.layer4.parameters():
    param.requires_grad = True

vae.resnet.layer3.requires_grad = True
for param in vae.resnet.layer3.parameters():
    param.requires_grad = True

vae.resnet.avgpool.requires_grad = True
for param in vae.resnet.avgpool.parameters():
    param.requires_grad = True

train_losses, test_losses = train(vae, train_loader, validation_loader, num_epoch=300, init_lr=0.001, lgamma = 0.984)


LR:  0.000984



Epoch: 1, Train loss: 129281.19945406627, Test loss: 140091.1599702381



LR:  0.000968256



Epoch: 2, Train loss: 128446.812311747, Test loss: 137698.13504464287



LR:  0.000952763904


...


LR:  0.0008648800628150849



Epoch: 9, Train loss: 127935.6220507028, Test loss: 136338.4025297619



LR:  0.0008510419818100435



Epoch: 10, Train loss: 127967.79433358433, Test loss: 136209.30208333334


KeyboardInterrupt: ignored

In [0]:
torch.save(vae.state_dict(), '/content/drive/My Drive/flowers_jpg/flower_GL4_final.torch.vae')

In [0]:
import hashlib
with open('/content/drive/My Drive/flowers_jpg/flower_GL4_final.torch.vae',"rb") as f:
    bytes = f.read() # read entire file as bytes
    readable_hash = hashlib.sha256(bytes).hexdigest();
    print(readable_hash)

011250eed4b56b4c2ae28b3619be002b28e4a7758bce8158352d376bc0d837fb
